In [1]:
import importlib

import training
import utils
import training_constants as tc
import preprocessing as pre
# Avaliar e selecionar o melhor seletor de variáveis
from sklearn.pipeline import Pipeline
import feature_selection as fs
import evaluation

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
importlib.reload(training)
importlib.reload(utils)
importlib.reload(tc)
importlib.reload(pre)
importlib.reload(fs)
importlib.reload(training)
importlib.reload(evaluation)


<module 'evaluation' from '/Users/patricia/Documents/code/python/behavior-detection/src/evaluation.py'>

In [3]:
import os
import warnings
import numpy as np

# Configurar variáveis de ambiente para não mostrar warning da Intel MKL
os.environ["MKL_CBWR"] = "AUTO"
os.environ["MKL_VERBOSE"] = "0"
os.environ["NUMPY_MKL_ERROR"] = "IGNORE"
os.environ["MKL_SERVICE_FORCE_INTEL"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["MKL_DYNAMIC"] = "FALSE"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["KMP_WARNINGS"] = "FALSE"

# Suprimir avisos de depreciação
warnings.filterwarnings("ignore", category=DeprecationWarning)

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /Users/patricia/anaconda3/envs/projetos_ML/include
    lib directory: /Users/patricia/anaconda3/envs/projetos_ML/lib
    name: mkl-sdl
    openblas configuration: unknown
    pc file directory: /Users/patricia/anaconda3/envs/projetos_ML/lib/pkgconfig
    version: '2023.1'
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep140221571962176
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -march=core2, -mtune=haswell, -mssse3, -ftree-vectorize, -fPIC, -fPIE, -fstack-protector-strong,
      -O2, -pipe, -isystem, /Users/patricia/anaconda3/envs/projetos_ML/include, -fdebug-prefix-map=/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b7iptlxgej/croot/numpy_and_numpy_base_1708638622773/work=/usr/local/src/conda/numpy-base-1.26.4,
      -fdebug-prefix-map=

In [4]:
X, y = pre.load_data("/Users/patricia/Documents/code/python/behavior-detection/data/new_logs_labels.csv")
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5525 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.2+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 5525 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5525 non-null   object
dtypes: object(1)
memory usage: 43.3+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [5]:
test_size = 0.2  # 80% for training, 20% for testing
X_train, X_test, y_train, y_test = pre.split_train_test_data(X, y, test_size, random_state=42)


Tamanho do conjunto de treino: 4420
Tamanho do conjunto de teste: 1105


In [6]:
import pandas as pd

print("Nro de instancias de cada classe em y_train e y_test:\n")

print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

Nro de instancias de cada classe em y_train e y_test:

ON TASK                 2528
ON SYSTEM                724
OFF TASK                 497
ON TASK CONVERSATION     335
ON TASK OUT              304
?                         32
Name: count, dtype: int64
ON TASK                 631
ON SYSTEM               183
OFF TASK                132
ON TASK CONVERSATION     79
ON TASK OUT              76
?                         4
Name: count, dtype: int64


In [7]:
y_train, label_encoder = pre.encode_labels(y_train)
y_test = label_encoder.transform(y_test)

In [8]:
# Pré-processar os dados uma vez
preprocessor = pre.create_preprocessor(X_train)
X_train = preprocessor.fit_transform(X_train)

In [9]:
best_selector, best_selector_name, selected_features = fs.evaluate_feature_selectors(X_train, y_train, n_features_to_select=10, n_components=10)

# Exibir o melhor seletor e as características selecionadas
print(f'O melhor seletor de variáveis é: {best_selector_name}')
if selected_features is not None:
    print('As características selecionadas são:')
    print(selected_features)



In [ ]:
# Adicionar o melhor seletor de variáveis no pipeline de treinamento
base_pipeline = Pipeline([
    ('feature_selection', best_selector)
])

In [ ]:

trained_models = training.train_model(X_train, y_train, tc.GRID_SEARCH, base_pipeline)

In [ ]:
dirpath = "/Users/patricia/Documents/code/python/behavior-detection/output/"

reports = evaluation.generate_reports(trained_models, X_train, y_train, X_test, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)